In [1]:
import leafmap

In [2]:
m = leafmap.Map()
m

Map(center=[20, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text…

In [3]:
m.add_basemap("Satellite")

In [4]:
import osmnx as ox

# Get a road network for a city or area
G = ox.graph_from_place("Angeles, Philippines", network_type="drive")  # options: 'all', 'walk', 'bike', 'drive'

# Convert to GeoDataFrame (edges = roads)
edges = ox.graph_to_gdfs(G, nodes=False)
edges.to_file('roads.shp')

In [5]:
import rasterio
from rasterio.mask import mask
import geopandas as gpd

img_tiff = "https://storage.googleapis.com/dpad-bucket/D2/smi/D2_SMI_2020-03-22T053207_2020-03-22T235959_L1C_COG.tif"
# This will be used for retrieving the metadata of the image(e.g. Bounds)

with rasterio.open(img_tiff) as src:
    bounds = src.bounds
    crs = src.crs

    roads = gpd.read_file('roads.shp')

    out_image, out_transform = mask(src, roads.geometry, invert=False, crop=True, nodata=0)
    out_meta = src.meta.copy()
    out_meta.update({
        "driver": "GTiff",
        "height": out_image.shape[1],
        "width": out_image.shape[2],
        "transform": out_transform,
        "crs": crs,
        "nodata": 0,
    })
with rasterio.open('roads_masked.tif', 'w', **out_meta) as dest:
    dest.write(out_image)

In [6]:
from localtileserver import TileClient, get_leaflet_tile_layer

client = TileClient('roads_masked.tif')
tile_layer = get_leaflet_tile_layer(client)
m.add_layer(tile_layer)

In [7]:
# Plot using leafmap
# m = leafmap.Map(center=[52.37, 4.89], zoom=12)
m.add_gdf(edges, layer_name="Roads", style={
  "color": "blue",       # Line color
  "weight": 2,           # Line thickness
  "opacity": 0.8         # Line opacity
})

In [6]:
import ee
import geemap

In [ ]:
geemap.ee_initialize()

In [9]:
gm = geemap.Map()

In [10]:
gm

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…